# Description

This notebook is similar to `40` (using LVs that we found to be significantly enriched for the lipids CRISPR analysis), but traits here are from eMERGE.

# Environment variables

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import conf

In [3]:
N_JOBS = conf.GENERAL["N_JOBS"]
display(N_JOBS)

3

In [4]:
%env MKL_NUM_THREADS=$N_JOBS
%env OPEN_BLAS_NUM_THREADS=$N_JOBS
%env NUMEXPR_NUM_THREADS=$N_JOBS
%env OMP_NUM_THREADS=$N_JOBS

env: MKL_NUM_THREADS=3
env: OPEN_BLAS_NUM_THREADS=3
env: NUMEXPR_NUM_THREADS=3
env: OMP_NUM_THREADS=3


# Modules

In [5]:
from pathlib import Path

import pandas as pd
from tqdm import tqdm

from gls import GLSPhenoplier

# Settings

In [6]:
OUTPUT_DIR = conf.RESULTS["GLS"]
display(OUTPUT_DIR)

OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/gls')

# Load data

## MultiPLIER summary

In [13]:
multiplier_model_summary = pd.read_pickle(conf.MULTIPLIER["MODEL_SUMMARY_FILE"])

In [14]:
multiplier_model_summary.shape

(2157, 5)

In [15]:
multiplier_model_summary.head()

,pathway,LV index,AUC,p-value,FDR
1,KEGG_LYSINE_DEGRADATION,1,0.388059,0.866078,0.956005
2,REACTOME_MRNA_SPLICING,1,0.733057,0.000048,0.000582
3,MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX,1,0.680555,0.001628,0.011366
4,KEGG_DNA_REPLICATION,1,0.549473,0.312155,0.539951
5,PID_MYC_ACTIVPATHWAY,1,0.639303,0.021702,0.083739


In [16]:
well_aligned_lvs = multiplier_model_summary[
    (multiplier_model_summary["FDR"] < 0.05) | (multiplier_model_summary["AUC"] >= 0.75)
]

display(well_aligned_lvs.shape)
display(well_aligned_lvs.head())

(469, 5)

,pathway,LV index,AUC,p-value,FDR
2,REACTOME_MRNA_SPLICING,1,0.733057,4.772691e-05,5.816211e-04
3,MIPS_NOP56P_ASSOCIATED_PRE_RRNA_COMPLEX,1,0.680555,1.628217e-03,1.136590e-02
8,REACTOME_MITOTIC_G1_G1_S_PHASES,1,0.686170,2.517619e-04,2.392292e-03
9,IRIS_Monocyte-Day0,2,0.890036,4.315812e-25,1.329887e-22
10,DMAP_MONO2,2,0.904676,1.313970e-16,1.574574e-14


In [17]:
well_aligned_lv_codes = set([f"LV{lvi}" for lvi in well_aligned_lvs["LV index"]])

In [18]:
len(well_aligned_lv_codes)

200

In [19]:
list(well_aligned_lv_codes)[:5]

['LV517', 'LV69', 'LV376', 'LV26', 'LV606']

## eMERGE traits info

In [20]:
# FIXME: in the future, there will be a specific entry in config for the eMERGE directory that should be replaced here
input_filepath = Path(
    conf.DATA_DIR,
    "emerge",
    "eMERGE_III_PMBB_GSA_v2_2020_phecode_AFR_EUR_cc50_counts_w_dictionary.txt",
).resolve()
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/data/emerge/eMERGE_III_PMBB_GSA_v2_2020_phecode_AFR_EUR_cc50_counts_w_dictionary.txt')

In [21]:
emerge_traits_df = pd.read_csv(
    input_filepath,
    sep="\t",
    dtype={"phecode": str},
    usecols=["phecode", "phenotype", "category"],
)

In [22]:
emerge_traits_df = emerge_traits_df.rename(
    columns={
        "phenotype": "phecode_phenotype",
        "category": "phecode_category",
    }
)

In [23]:
emerge_traits_df.shape

(309, 3)

In [24]:
emerge_traits_df.head()

,phecode,phecode_phenotype,phecode_category
0,008,Intestinal infection,infectious diseases
1,008.5,Bacterial enteritis,infectious diseases
2,008.52,Intestinal infection due to C. difficile,infectious diseases
3,038,Septicemia,infectious diseases
4,038.3,Bacteremia,infectious diseases


## eMERGE (S-MultiXcan)

In [25]:
# FIXME: in the future, there will be a specific entry in config for the eMERGE directory that should be replaced here
emerge_smultixcan_zscores_filepath = Path(
    conf.DATA_DIR,
    "emerge",
    "gene_assoc",
    "emerge-smultixcan-mashr-zscores.pkl",
).resolve()

display(emerge_smultixcan_zscores_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/data/emerge/gene_assoc/emerge-smultixcan-mashr-zscores.pkl')

In [26]:
_tmp = pd.read_pickle(emerge_smultixcan_zscores_filepath)

In [27]:
_tmp.shape

(22198, 309)

In [28]:
_tmp.head()

,292.3,079,741,418.1,280,747,591,426.21,244,070.3,...,427,585.4,345,366.2,420.2,195,250.2,250.1,371,562
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,1.043436,1.134028,1.595979,0.399451,0.725100,1.655468,1.803598,0.125250,1.053218,0.922592,...,0.364789,0.453873,0.759558,1.173671,0.298944,1.029646,0.826238,0.685711,2.069701,1.544766
ENSG00000000457,1.236752,0.119837,0.224120,0.200610,1.304190,0.541478,0.646474,0.493759,0.189680,1.563273,...,1.519500,0.201290,1.020996,0.592006,0.632661,0.995453,0.069362,0.460979,0.217046,1.350475
ENSG00000000460,0.497108,0.005690,0.317710,0.184918,1.136574,1.708245,0.288962,1.070026,0.403510,1.106321,...,0.678253,1.166204,1.138126,0.608565,0.449551,0.096689,0.887467,0.143815,0.236647,1.048859
ENSG00000000938,0.374442,1.099899,1.975795,0.107667,0.903520,0.673819,0.855749,0.499088,0.375406,0.815757,...,1.585687,0.584143,0.751384,2.758230,0.572712,0.141023,1.437022,0.288943,0.665727,1.225229
ENSG00000000971,1.108762,1.133163,0.227188,0.222948,0.072709,0.256124,0.540911,0.606947,0.229133,1.557433,...,2.349373,1.122458,0.155346,0.990123,0.326914,1.100210,1.099537,2.511358,0.950262,1.467305


## GLS results on PhenomeXcan

In [29]:
input_filepath = conf.RESULTS["GLS"] / "gls_phenotypes-crispr_lvs.pkl"
display(input_filepath)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/gls/gls_phenotypes-crispr_lvs.pkl')

In [30]:
gls_phenomexcan_crispr = pd.read_pickle(input_filepath)

In [31]:
gls_phenomexcan_crispr.shape

(2142, 9)

In [32]:
gls_phenomexcan_crispr.head()

,part_k,cluster_id,phenotype,lv,lv_set,lv_with_pathway,coef,pvalue,summary
0,29,17,20003_1140861958-Treatmentmedication_code_simv...,LV120,lipids-increasing,False,0.011685,0.345237,GLS Regression Res...
1,29,17,20003_1140861958-Treatmentmedication_code_simv...,LV122,lipids-increasing,False,-0.004370,0.723659,GLS Regression Res...
2,29,17,20003_1140861958-Treatmentmedication_code_simv...,LV131,lipids-decreasing,False,-0.000511,0.966819,GLS Regression Res...
3,29,17,20003_1140861958-Treatmentmedication_code_simv...,LV157,lipids-decreasing,False,0.010968,0.352087,GLS Regression Res...
4,29,17,20003_1140861958-Treatmentmedication_code_simv...,LV199,lipids-decreasing,False,-0.009505,0.435201,GLS Regression Res...


# Select LV from previous GLS run on PhenomeXcan

This selectes the LVs enriched for the lipids CRISPR anaysis.

In [37]:
gls_phenomexcan_lvs = (
    gls_phenomexcan_crispr[["lv", "lv_set"]].drop_duplicates().reset_index(drop=True)
)

In [38]:
gls_phenomexcan_lvs.shape

(51, 2)

In [39]:
gls_phenomexcan_lvs.head()

,lv,lv_set
0,LV120,lipids-increasing
1,LV122,lipids-increasing
2,LV131,lipids-decreasing
3,LV157,lipids-decreasing
4,LV199,lipids-decreasing


# Select relevant traits from eMERGE

Here we don't have partitions/cluters as with PhenomeXcan. So for this analysis (LVs related to lipids), I select relevant categories of traits from eMERGE (we have a "category" column for this results).

In [40]:
emerge_traits_df["phecode_category"].unique()

array(['infectious diseases', 'neoplasms', 'endocrine/metabolic',
       'hematopoietic', 'mental disorders', 'neurological',
       'sense organs', 'circulatory system', 'respiratory', 'digestive',
       'genitourinary', 'dermatologic', 'musculoskeletal',
       'congenital anomalies'], dtype=object)

In [41]:
gls_traits = emerge_traits_df[
    emerge_traits_df["phecode_category"].isin(
        [
            "circulatory system",
            "endocrine/metabolic",
        ]
    )
]["phecode"].unique()

In [42]:
gls_traits.shape

(128,)

In [43]:
gls_traits

array(['241', '241.1', '241.2', '244', '244.2', '244.4', '249', '250',
       '250.1', '250.2', '250.22', '250.24', '250.4', '250.41', '250.6',
       '260', '260.2', '261', '261.4', '269', '270', '272', '272.1',
       '272.11', '272.13', '274', '274.1', '274.11', '275', '275.3',
       '275.5', '275.53', '276', '276.1', '276.12', '276.13', '276.14',
       '276.4', '276.41', '276.5', '276.6', '277.7', '278', '278.1',
       '278.11', '279', '394', '394.2', '394.7', '395', '395.1', '395.2',
       '401', '401.1', '401.2', '401.21', '401.22', '411', '411.1',
       '411.2', '411.3', '411.4', '411.8', '414', '415', '415.1',
       '415.11', '415.2', '415.21', '416', '418', '418.1', '420', '420.2',
       '420.3', '425', '425.1', '425.2', '426', '426.2', '426.21',
       '426.24', '426.3', '426.31', '426.32', '427', '427.1', '427.11',
       '427.12', '427.2', '427.21', '427.22', '427.4', '427.5', '427.6',
       '427.8', '427.9', '428', '428.1', '428.2', '428.3', '428.4', '429',
       

# GLSPhenoplier

## Get list of phenotypes/lvs pairs

In [44]:
phenotypes_lvs_pairs = []

for idx, lv_row in gls_phenomexcan_lvs.iterrows():
    for phenotype_code in gls_traits:
        phenotypes_lvs_pairs.append(
            {
                "phenotype": phenotype_code,
                "lv": lv_row["lv"],
                "lv_set": lv_row["lv_set"],
            }
        )

phenotypes_lvs_pairs = pd.DataFrame(phenotypes_lvs_pairs).drop_duplicates()

In [45]:
phenotypes_lvs_pairs = phenotypes_lvs_pairs.sort_values("phenotype").reset_index(
    drop=True
)

In [46]:
phenotypes_lvs_pairs.shape

(6528, 3)

In [47]:
phenotypes_lvs_pairs.head()

,phenotype,lv,lv_set
0,241,LV120,lipids-increasing
1,241,LV783,lipids-decreasing
2,241,LV621,lipids-increasing
3,241,LV494,lipids-increasing
4,241,LV246,lipids-increasing


## Run

In [48]:
output_file = OUTPUT_DIR / "gls_phenotypes-emerge-crispr_lvs.pkl"
display(output_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/gls/gls_phenotypes-emerge-crispr_lvs.pkl')

In [49]:
results = []

pbar = tqdm(total=phenotypes_lvs_pairs.shape[0])

for idx, row in phenotypes_lvs_pairs.iterrows():
    phenotype_code = row["phenotype"]
    lv_code = row["lv"]

    pbar.set_description(f"{phenotype_code} - {lv_code}")

    gls_model = GLSPhenoplier(
        smultixcan_result_set_filepath=emerge_smultixcan_zscores_filepath
    )
    gls_model.fit_named(lv_code, phenotype_code)
    res = gls_model.results

    results.append(
        {
            "phenotype": phenotype_code,
            "lv": lv_code,
            "lv_set": row["lv_set"],
            "lv_with_pathway": lv_code in well_aligned_lv_codes,
            "coef": res.params.loc["lv"],
            "pvalue": res.pvalues.loc["lv"],
            "summary": gls_model.results_summary,
        }
    )

    # save results every 10 models trained
    if (idx % 10) == 0:
        pd.DataFrame(results).to_pickle(output_file)

    pbar.update(1)

pbar.close()

458.9 - LV99: 100%|██████████| 6528/6528 [17:34:22<00:00,  9.69s/it]


In [50]:
results = pd.DataFrame(results)

In [51]:
results.shape

(6528, 7)

In [52]:
results.head()

,phenotype,lv,lv_set,lv_with_pathway,coef,pvalue,summary
0,241,LV120,lipids-increasing,False,0.021088,0.090966,GLS Regression Res...
1,241,LV783,lipids-decreasing,False,-0.022565,0.073047,GLS Regression Res...
2,241,LV621,lipids-increasing,False,-0.017751,0.166719,GLS Regression Res...
3,241,LV494,lipids-increasing,False,0.027091,0.029469,GLS Regression Res...
4,241,LV246,lipids-increasing,True,0.004652,0.710773,GLS Regression Res...


In [53]:
results.sort_values("pvalue").head(10)

,phenotype,lv,lv_set,lv_with_pathway,coef,pvalue,summary
1947,276.41,LV246,lipids-increasing,True,0.054395,0.000016,GLS Regression Res...
3008,411.1,LV157,lipids-decreasing,False,0.050759,0.000021,GLS Regression Res...
1903,276.4,LV246,lipids-increasing,True,0.053259,0.000024,GLS Regression Res...
4123,426.21,LV199,lipids-decreasing,False,0.048702,0.000079,GLS Regression Res...
2653,401,LV157,lipids-decreasing,False,0.046407,0.000118,GLS Regression Res...
2561,395.1,LV621,lipids-increasing,False,0.048892,0.000131,GLS Regression Res...
530,250.22,LV415,lipids-increasing,False,0.046220,0.000162,GLS Regression Res...
1519,275.3,LV678,lipids-decreasing,True,0.045819,0.000168,GLS Regression Res...
2733,401.1,LV157,lipids-decreasing,False,0.045354,0.000172,GLS Regression Res...
5351,429.2,LV469,lipids-decreasing,False,0.044753,0.000179,GLS Regression Res...


## Save

In [54]:
results.to_pickle(output_file)